In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from urllib.request import urlopen
from netCDF4 import Dataset
import requests
from io import BytesIO
import dask
import cmocean
import calendar
import seaborn as sns

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
cartopy.config['data_dir'] = os.getenv('CARTOPY_DIR', cartopy.config.get('data_dir'))

from matplotlib import pyplot as plt
#%config InlineBackend.figure_format = 'retina'
plt.ion()  # To trigger the interactive inline mode

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [2]:
url="https://data-cbr.csiro.au/thredds/dodsC/catch_all/CMAR_CAWCR-Wave_archive/CAWCR_Wave_Hindcast_aggregate/gridded/ww3.pac_4m.202401.nc"
ds = xr.open_dataset(url)
ds

# https://data-cbr.csiro.au/thredds/dodsC/catch_all/CMAR_CAWCR-Wave_archive/CAWCR_Wave_Hindcast_aggregate/gridded/ww3.aus_4m.197901.nc

<xarray.Dataset>
Dimensions:    (longitude: 1276, latitude: 751, time: 744)
Coordinates:
  * longitude  (longitude) float32 125.0 125.1 125.1 125.2 ... 209.9 209.9 210.0
  * latitude   (latitude) float32 -30.0 -29.93 -29.87 -29.8 ... 19.87 19.93 20.0
  * time       (time) datetime64[ns] 2024-01-01 ... 2024-01-31T23:00:00.00001...
Data variables: (12/61)
    MAPSTA     (latitude, longitude) int16 ...
    dpt        (time, latitude, longitude) float32 ...
    uwnd       (time, latitude, longitude) float32 ...
    vwnd       (time, latitude, longitude) float32 ...
    ice        (time, latitude, longitude) float32 ...
    hs         (time, latitude, longitude) float32 ...
    ...         ...
    vtus       (time, latitude, longitude) float32 ...
    uuss       (time, latitude, longitude) float32 ...
    vuss       (time, latitude, longitude) float32 ...
    mssx       (time, latitude, longitude) float32 ...
    mssy       (time, latitude, longitude) float32 ...
    dtd        (time, latitude, longitude) float32 ...
Attributes: (12/19)
    WAVEWATCH_III_version_number:           4.18b
    WAVEWATCH_III_switches:                 F90 NOGRB LRB4 DIST MPI NOPA UQ P...
    SIN4_namelist_parameter_BETAMAX:        1.33
    SDS4_namelist_parameter_WHITECAPWIDTH:  0.3
    product_name:                           ww3.pac_4m.202401.nc
    area:                                   Pacific 4 min wave grid
    ...                                     ...
    maximum_altitude:                       9000 m
    altitude_resolution:                    n/a
    start_date:                             2024-01-01T00:00:00Z
    stop_date:                              2024-01-31T23:00:00Z
    DODS_EXTRA.Unlimited_Dimension:         time
    EXTRA_DIMENSION.level:                  1

In [3]:
# clip by coordinates (x,y)

min_lon = 153.06     # lower left longitude
min_lat = -18.26     # lower left latitude
max_lon = 154.27     # upper right longitude
max_lat = -17.2     # upper right latitude

# Defining the boundaries
lon_bnds = [min_lon, max_lon]
lat_bnds = [min_lat, max_lat]

# Performing the reduction
ds_clip = ds.sel(latitude=slice(*lat_bnds), longitude=slice(*lon_bnds))
ds_clip

<xarray.Dataset>
Dimensions:    (longitude: 19, latitude: 15, time: 744)
Coordinates:
  * longitude  (longitude) float32 153.1 153.1 153.2 153.3 ... 154.1 154.2 154.3
  * latitude   (latitude) float32 -18.2 -18.13 -18.07 ... -17.4 -17.33 -17.27
  * time       (time) datetime64[ns] 2024-01-01 ... 2024-01-31T23:00:00.00001...
Data variables: (12/61)
    MAPSTA     (latitude, longitude) int16 ...
    dpt        (time, latitude, longitude) float32 ...
    uwnd       (time, latitude, longitude) float32 ...
    vwnd       (time, latitude, longitude) float32 ...
    ice        (time, latitude, longitude) float32 ...
    hs         (time, latitude, longitude) float32 ...
    ...         ...
    vtus       (time, latitude, longitude) float32 ...
    uuss       (time, latitude, longitude) float32 ...
    vuss       (time, latitude, longitude) float32 ...
    mssx       (time, latitude, longitude) float32 ...
    mssy       (time, latitude, longitude) float32 ...
    dtd        (time, latitude, longitude) float32 ...
Attributes: (12/19)
    WAVEWATCH_III_version_number:           4.18b
    WAVEWATCH_III_switches:                 F90 NOGRB LRB4 DIST MPI NOPA UQ P...
    SIN4_namelist_parameter_BETAMAX:        1.33
    SDS4_namelist_parameter_WHITECAPWIDTH:  0.3
    product_name:                           ww3.pac_4m.202401.nc
    area:                                   Pacific 4 min wave grid
    ...                                     ...
    maximum_altitude:                       9000 m
    altitude_resolution:                    n/a
    start_date:                             2024-01-01T00:00:00Z
    stop_date:                              2024-01-31T23:00:00Z
    DODS_EXTRA.Unlimited_Dimension:         time
    EXTRA_DIMENSION.level:                  1

In [ ]:
# use chatGPT to make a new code, based on this one, that will loop throught the years and output monthly averaged for 10 years 
# without the need to input eah year (like in this code)

# month_st = 1   # Starting month 
# month_ed = 3   # Ending month 
# year = 2018    # Year

# Based on the server the file naming convention 
# biofiles = [f"{base_url}{year}-{month:02}.nc" for month in range(month_st, month_ed+1)]
# hydrofiles = [f"{base_url2}{year}-{month:02}.nc" for month in range(month_st, month_ed+1)]
# biofiles



In [6]:
# years 1979-1990

# Define the base URLs for the datasets
base_url_2024 = "https://data-cbr.csiro.au/thredds/dodsC/catch_all/CMAR_CAWCR-Wave_archive/CAWCR_Wave_Hindcast_aggregate/gridded/ww3.pac_4m."

# Define the range of years
start_year = 1979
end_year = 1980

# Loop through the years
for year in range(start_year, end_year + 1):
    # Initialize lists to store monthly averaged hs values
    monthly_averages = []

    # Loop through the months
    for month in range(1, 13):
        # Construct the file URLs for both datasets
        file_url_2024 = f"{base_url_2024}{2024}{month:02}.nc"

        # Open the datasets
        ds_2024 = xr.open_dataset(file_url_2024)

        # Extract the 'hs' variable from both datasets
        hs_2024 = ds_2024['hs']

        # Calculate the monthly average of 'hs' for both datasets
        monthly_average_pac_4m = hs_2024.mean().values

        # Append the monthly average to the list
        monthly_averages.append(monthly_average_pac_4m)

        # Close the datasets to free up memory
        ds_2024.close()

    # Print or store the monthly averages for the current year
    print(f"Monthly averages for {year}: {monthly_averages}")

oc_open: server error retrieving url: code=403 message="Request too big=1425.914688 Mbytes, max=500.0"oc_open: server error retrieving url: code=403 message="Request too big=1425.914688 Mbytes, max=500.0"oc_open: server error retrieving url: code=403 message="Request too big=1425.914688 Mbytes, max=500.0"oc_open: server error retrieving url: code=403 message="Request too big=1425.914688 Mbytes, max=500.0"oc_open: server error retrieving url: code=403 message="Request too big=1425.914688 Mbytes, max=500.0"oc_open: server error retrieving url: code=403 message="Request too big=1425.914688 Mbytes, max=500.0"oc_open: server error retrieving url: code=403 message="Request too big=1425.914688 Mbytes, max=500.0"

RuntimeError: NetCDF: Authorization failure

In [ ]:
# years 1990 - 2023

# Define the base URLs for the datasets
base_url_2024 = "https://data-cbr.csiro.au/thredds/dodsC/catch_all/CMAR_CAWCR-Wave_archive/CAWCR_Wave_Hindcast_aggregate/gridded/ww3.pac_4m."

# Define the range of years
start_year = 1990
end_year = 2023

# Loop through the years
for year in range(start_year, end_year + 1):
    # Initialize lists to store monthly averaged hs values
    monthly_averages = []

    # Loop through the months
    for month in range(1, 13):
        # Construct the file URLs for both datasets
        file_url_2024 = f"{base_url_2024}{2024}{month:02}.nc"

        # Open the datasets
        ds_2024 = xr.open_dataset(file_url_2024)

        # Extract the 'hs' variable from both datasets
        hs_2024 = ds_2024['hs']

        # Calculate the monthly average of 'hs' for both datasets
        monthly_average_pac_4m = hs_2024.mean().values

        # Append the monthly average to the list
        monthly_averages.append(monthly_average_pac_4m)

        # Close the datasets to free up memory
        ds_2024.close()

    # Print or store the monthly averages for the current year
    print(f"Monthly averages for {year}: {monthly_averages}")